In [3]:
pip install catboost

                                              0.0/101.0 MB ? eta -:--:--
                                              0.3/101.0 MB 8.3 MB/s eta 0:00:13
                                              0.7/101.0 MB 8.4 MB/s eta 0:00:12
                                              1.2/101.0 MB 9.6 MB/s eta 0:00:11
                                              1.7/101.0 MB 9.6 MB/s eta 0:00:11
                                              2.2/101.0 MB 9.9 MB/s eta 0:00:10
     -                                        2.6/101.0 MB 9.9 MB/s eta 0:00:10
     -                                        3.1/101.0 MB 9.8 MB/s eta 0:00:10
     -                                        3.6/101.0 MB 9.9 MB/s eta 0:00:10
     -                                       4.1/101.0 MB 10.0 MB/s eta 0:00:10
     -                                        4.2/101.0 MB 9.4 MB/s eta 0:00:11
     -                                        4.5/101.0 MB 9.0 MB/s eta 0:00:11
     -                                        4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install six

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install ipywidgets

                                              0.0/139.3 kB ? eta -:--:--
     -------------------------------------- 139.3/139.3 kB 4.2 MB/s eta 0:00:00
                                              0.0/2.3 MB ? eta -:--:--
     --------                                 0.5/2.3 MB 9.8 MB/s eta 0:00:01
     -----------------                        1.0/2.3 MB 10.3 MB/s eta 0:00:01
     ------------------------                 1.4/2.3 MB 9.7 MB/s eta 0:00:01
     -----------------------------            1.7/2.3 MB 9.1 MB/s eta 0:00:01
     -------------------------------------    2.2/2.3 MB 9.2 MB/s eta 0:00:01
     ---------------------------------------  2.3/2.3 MB 9.1 MB/s eta 0:00:01
     ---------------------------------------  2.3/2.3 MB 9.1 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 6.1 MB/s eta 0:00:00
                                              0.0/215.0 kB ? eta -:--:--
     -------------------------------------- 215.0/215.0 kB 6.6 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import unicodedata
from pathlib import Path

import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from category_encoders import CountEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline


In [8]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sub_df = pd.read_csv("submit_sample.csv")

In [10]:
train_df.shape

(27532, 16)

In [11]:
test_df.shape

(27537, 15)

In [12]:
sub_df.shape

(27536, 2)

In [13]:
train_df.head(5)

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158


In [15]:
train_df.tail(5)

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,rwd,full-size,pickup,black,NaN,5400
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054
27531,27531,orange county,2009,chevrolet,excellent,8 cylinders,gas,91813,rebuilt,automatic,rwd,full-size,van,white,ca,20801


In [16]:
test_df.head(5)

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553,clean,automatic,fwd,full-size,SUV,red,NaN
1,27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385,salvage,automatic,fwd,mid-size,sedan,black,or
2,27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489,clean,automatic,fwd,full-size,sedan,black,oh
3,27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310,clean,automatic,4wd,mid-size,SUV,red,co
4,27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839,rebuilt,automatic,4wd,mid-size,SUV,silver,nc


In [17]:
test_df.tail(5)

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
27532,55064,great falls,2016,gmc,excellent,6 cylinders,gas,90902,rebuilt,automatic,fwd,full-size,truck,black,mt
27533,55065,las vegas,2012,nissan,excellent,4 cylinders,gas,27234,rebuilt,automatic,fwd,mid-size,sedan,white,nv
27534,55066,phoenix,2002,bmw,excellent,6 cylinders,gas,99761,clean,automatic,rwd,full-size,coupe,blue,az
27535,55067,colorado springs,2006,bmw,excellent,6 cylinders,gas,162279,clean,automatic,4wd,mid-size,sedan,silver,co
27536,55068,syracuse,2006,jeep,excellent,6 cylinders,gas,186965,clean,automatic,4wd,full-size,SUV,black,ny
